# 06 · Estimación OK con vecindad anisotrópica

Kriging ordinario con vecindad anisotrópica, octantes y máximo por barreno.

In [ ]:
import os, sys
import numpy as np
import pandas as pd

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()
    os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.geostats_pipeline.block_model import BlockDiscretization, discretize_block
from src.geostats_pipeline.kriging import SearchParameters, ordinary_kriging

df = pd.read_csv('data_sample/sample_drillholes.csv')
df = df.rename(columns={'X': 'x', 'Y': 'y', 'Z': 'z', 'grade': 'grade'})
df['drillhole'] = df['domain']
df.head()

## 1. Parámetros de variograma y vecindad

In [ ]:
variogram_model = {
    'nug': 0.1,
    'cc1': 0.9,
    'it1': 1,
    'hmaj1': 150.0,
    'cc2': 0.0,
    'it2': 1,
    'hmaj2': 150.0,
}

search = SearchParameters(
    ranges=(150.0, 100.0, 50.0),
    angles=(30.0, 0.0, 0.0),
    min_samples=6,
    max_samples=16,
    octants=8,
    max_per_drillhole=3,
    drillhole_col='drillhole',
)
search

## 2. Estimación puntual

In [ ]:
target = (df['x'].mean(), df['y'].mean(), df['z'].mean())
result_point = ordinary_kriging(
    df, 'x', 'y', 'z', 'grade', target, variogram_model, search
)
result_point

## 3. Estimación de bloque

In [ ]:
block_size = (50.0, 50.0, 10.0)
discretization = BlockDiscretization(nx=3, ny=3, nz=2)
block_points = discretize_block(target, block_size, discretization)

result_block = ordinary_kriging(
    df, 'x', 'y', 'z', 'grade', target, variogram_model, search, block_points=block_points
)
result_block